# Router Engine

Given a query, the router will pick what to run.  This shows dynamic query understanding.

## Setup

In [19]:
# this is required since Juypyter notebook is not async by default.
import nest_asyncio
nest_asyncio.apply()

## Ingestion pipeline

### 1. Load the data

In [20]:
from llama_index.core import SimpleDirectoryReader

lightning_ai="/teamspace/studios/this_studio/production-incentives-insider/data/sample"
documents = SimpleDirectoryReader(lightning_ai).load_data()

### 2. Use a text splitter to Split Documents

In [4]:
from llama_index.core.node_parser import SentenceSplitter

text_parser = SentenceSplitter(chunk_size=1024)

### 3. Construct Nodes from Text Chunks

In [6]:
nodes = text_parser.get_nodes_from_documents(documents)

### 4. Generate Embeddings for Each Node

In [21]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## Define Indexes

- Summary Index 
- Vector Index 

Think of the index as a set of metadata over the data.  Different indexes have different index behavior.  A vector index indexes node through text embeddings.  Querying a vector index in llamaindex will return the most similar nodes. A summary index is also a simple index, and a query to it will return all of the nodes that are currently in the index.

In [9]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## Turn the indexes into query engines

Each query engine represents an interface over the data that has been stored in the index.  It combines retrieval with LLM synthesis.  Each engine is specific for a certain kind of question.  This is where the use case of the router becomes significant.  

In [10]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)

vector_query_engine = vector_index.as_query_engine()

## Create query tools

The query tools are simply the query engine with additional metadata.  The metadata reflects the kind of information the tool can answer.  For example, asking it to "summarize" the article would launch the summary_tool.

In [11]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to the document."
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the document."
    ),
)

## Define Router

Llamaindex provides several diffent selectors.  

- The LLM selectors use the LLM to output a JSON that is parsed, and corresponding indexes are queried.
- The Pydantic selects use the OpenAI Function Calling API to produce pydantic selection objects rather than raw JSON.

In this example we are using the LLMSingleSelector. The RouterQueryEngine takes in a selector type as well as a set of tools.

In [12]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    # allows us to view the intermediate steps of the query engine
    verbose=True,
)

In [9]:
response = query_engine.query("Summarize the economic contributions to the production of Yellowstone.")
print(str(response))

Selecting query engine 0: Summarization questions related to economic contributions to the production of Yellowstone.
The economic contributions to the production of Yellowstone encompass a wide range of sectors, including screen production-specific expenses, business support services, construction costs, digital services, real estate rentals, travel and transport expenses, hospitality and catering services, finance and legal services, fashion and beauty expenditures, music and performing arts costs, power and utilities usage, safety and security expenses, training and education investments, health and medical services, and miscellaneous local labor costs. These contributions demonstrate the extensive impact of the production on various industries, highlighting a significant economic ripple effect throughout the regions where filming takes place.


In [10]:
print(len(response.source_nodes))

148
